# Análisis Exploratorio y Preparación de Datos
## TC 2031 Ciencia de Datos
## @etnarojas


- Objetivo:
   
    - Mostrar lo que se hace cuando se hace un análisis exploratorio de los datos y se preparan para poder usarlos para hacer análisis o modelos.
    - En proyectos reales entre el 50% y 80% del tiempo se emplea en tareas de preparación de los datos para empezar a modelar
    
  
- Librerías de Python usadas:
    - pandas
    - Matplotlib
    - Numpy
   

Dataset para el ejemplo tomado de : https://archive.ics.uci.edu/ml/datasets/Adult

1. Limpieza de Datos
    1. Manejo de diferentes tipos de datos 
    2. Manejo de datos faltantes
2. Exploración de los Datos
    1. Descripción de los datos
    2. Gráficas de las distribuciones de los Datos
    3. Detección de valores atípicos
3. Creando los Datos para hacer el Modelo
    1. Relaciones entre variables
    2. Selección de variables

## Modelo

Un modelo que dados los atributos sobre una persona haga una predicción para saber s si su ingreso es <= 50000 o > 50000 Dlls

## Datos
### Una descripción del tipo de dato y que representa 

- age: continuous.
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt: continuous.represents final weight, which is the number of units in the target population that the responding unit represents
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex: Female, Male.
- capital-gain: continuous.
- capital-loss: continuous.
- hours-per-week: continuous.
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.7



In [ ]:
# Importando librerías y datos
import numpy as np
import pandas as pd

df = pd.read_csv('adult.csv',na_values=['#NAME?'])

In [ ]:
print(df.head(5))


In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
type(df['education'][0])

In [ ]:
# Ver la variable 'income' el tipo y cuantos hay de cada tipo
print(df['income'].value_counts())

In [ ]:
# Asignar el valor de 0 Si income <=50K y de 1 Si 1 income >50K
df['income'] = [0 if x == '<=50K' else 1 for x in df['income']]

# Crear dos DataFrames X con variables indpendientes o features y uno y con variable dependiente , la que vamos a predecir
X = df.drop('income', 1)
y = df.income

In [ ]:
print(X.head(5))

In [ ]:
print(y.head(5))

## 1. Limpieza de Datos

### A. Manejo de diferentes tipos de datos 

- Hay tres tipos principales de datos:
    - Cuantitativo, Numérico: ingreso, edad
    - Cualitativo, Categórico:  género, nacionalidad
    - Cualitativo, Categórico Ordinal: bajo medio alto
    
- Los modelos sólo pueden manejar variables cuantitativas numéricas
- Se deben convertir las variables cualitativas ó categóricas en variables numéricas
    - Crear variables extra
    - Transformar una variable cualitativa o categórica en un conjunto de variables extra, cada una representando una categoría única
    - En el conjunto de variables extra, 1 indica que la observación pertenece a esa categoría
    

In [ ]:
# Education es una variable cualitativa
print(X['education'].head(20))

In [ ]:
# Ver las caterorías de las variables categoricas o cualitativas

for col_name in X.columns:
    if X[col_name].dtypes == 'object':
        unique_cat = len(X[col_name].unique())
        print('Variable ''{col_name} tiene {unique_cat} categorías'.format(col_name=col_name, unique_cat=unique_cat))


In [ ]:
# Use get_dummies de pandas para crear las variables dummies extra para representar a las variable cualitativa como numéricas
# Otra opción es usar  OneHotEncoder de la librería:  sci-kit learn
print(pd.get_dummies(X['education']).head(5))

In [ ]:
# Para ver las categorías de 'native_country' y vemos que es dominante la de Unites States
print(X['native_country'].value_counts().sort_values(ascending=False).head(10))

In [ ]:
# Podemos hacer que la baja frecuencia de las otras queden clasificadas como "Other"
X['native_country'] = ['United-States ' if x == 'United-States' else 'Other' for x in X['native_country']]

print(X['native_country'].value_counts().sort_values(ascending=False))

In [ ]:
# Crear toda la lista de variables categóricas para transformarlas a numéricas
todummy_list = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

In [ ]:
# Función para llamar get dummies c
def dummy_df(df, todummy_list):
    for x in todummy_list:
        dummies = pd.get_dummies(df[x], prefix=x, dummy_na=False)
        df = df.drop(x, 1)
        df = pd.concat([df, dummies], axis=1)
    return df

In [ ]:
X = dummy_df(X, todummy_list)
print(X.head(5))

### B. Manejo de datos faltantes

- Los modelos no pueden manejar datos faltantes


- La solución más simple
    - Eliminar observaciones / características que tienen datos faltantes
    

- Pero, eliminar datos faltantes puede presentar muchos problemas
    - Datos faltantes están distribuidos al azar y son muchos : posiblemente pierda muchos de sus datos
    - Datos faltantes no están distribuidos al azar y son muchos : además de perder datos, también está introduciendo posibles sesgos


- Una solución alternativa es utilizar la imputación.
    - Reemplace el valor faltante con otro valor
    - Estrategias: media, mediana, valor de frecuencia más alta de una variable dada
    
  

In [ ]:
# Cuantos son datos faltantes?
X.isnull().sum().sort_values(ascending=False).head()

In [ ]:
# Para Imputar / Reemplazar los valores faltantes se usa Imputer de sklearn.preprocessing
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values='NaN', strategy='median', axis=0)
imp.fit(X)
X = pd.DataFrame(data=imp.transform(X) , columns=X.columns)

In [ ]:
#Ahora volvemos a verificar como los nulos se han reemplazado con un valor
X.isnull().sum().sort_values(ascending=False).head()

## 2. Exploration de los Datos
- Entender el problema y los datos es extremadamente importante para construir modelos
- Se hace lo que se conoce como Análisis Exploratorio de los Datos para explorar sus datos entenderlos y tomar mejores decisiones al modelar

### A.Descripción de los Datos

In [ ]:
X.describe()

### B.Gráficas de las distribuciones de los Datos

In [ ]:
# Use pyplot de matplotlib para hacer histograms
%matplotlib inline
import matplotlib.pyplot as plt

def plot_histogram(x):
    plt.hist(x, color='gray', alpha=0.5)
    plt.title('Histogram of ''{var_name}'.format(var_name=x.name))
    plt.xlabel('Value')
    plt.ylabel('Frecuency')
    plt.show()

In [ ]:
plot_histogram(X['age'])

In [ ]:
for col_name in X.columns:
    plot_histogram(X[col_name])
    

### C. Detección de valores atípicos

- Un valor atípico es una observación que se desvía drásticamente de otras observaciones en un conjunto de datos


- Ocurrencia:
    - Natural, por ejemplo Ingresos de Millonarios
    - Error, por ejemplo peso humano de 5,00 kg. debido al error de escritura adicional 0

- ¿Por qué son problemáticos?
    - Si ocurren naturalmente
        - No necesariamente problemático
        - Pero pueden sesgar su modelo
    - Error 
        - Indicativo de problemas de calidad de datos.
        - Tratar de la misma manera como un valor faltante, es decir, usar la imputación
   
   
- ¿Cómo se detectan?
    - Visualizaciones Box Plot



 ### Detección de valores atípicos - Usando Box Plots
- Identifica valores extremos en los datos.
- Los valores atípicos se definen como:
    - Valores inferiores a Q1-1.5 (Q3-Q1) o superiores a Q3 + 1.5 (Q3-Q1)

 

In [ ]:
def find_outliers_tukey(x):
    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3-q1 
    floor = q1 - 1.5*iqr
    ceiling = q3 + 1.5*iqr
    outlier_indices = list(x.index[(x < floor)|(x > ceiling)])
    outlier_values = list(x[outlier_indices])

    return outlier_indices, outlier_values

In [ ]:
from IPython.display import Image
Image(filename='outliers_boxplot.jpg')
# Imagine taken from: 
# http://datapigtechnologies.com/blog/index.php/highlighting-outliers-in-your-data-with-the-tukey-method/

In [ ]:
tukey_indices, tukey_values = find_outliers_tukey(X['age'])
print(np.sort(tukey_values))

In [ ]:
from IPython.display import Image
Image(filename='outliers.jpg')

## 3. Creando los Datos para hacer el Modelo

### A. Relaciones entre variables

- La variables independientes pueden estar correlacionadas y en ocasiones hay modelos como las regersiones que no permiten isar variables correlacionadas porque se afecta el resultado.

- Fecha de Nacimiento y Edad tienen una relación entre ellas y hay que decidir cual usar.
- No siempre es evidente esta relación entre las variables




In [ ]:
# Puedes analizar una matriz de correlaciones de todas las variables
X.corr()

### B. Selección de Variables


In [ ]:
# Al agregar variable dummies por variables categóricas se incrementó el número de variables
print(df.shape)
print(X.shape)

In [ ]:
# Usar train_test_split in sklearn.cross_validation para partir los datos en dos set de enternamiento y de testeo
import warnings
warnings.filterwarnings("ignore")
from sklearn.cross_validation import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=1)


In [ ]:

# Usar el  método feature selection para seleccionar las varuables más significativas
import sklearn.feature_selection


select = sklearn.feature_selection.SelectKBest(k=20)
selected_features = select.fit(X_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X.columns[i] for i in indices_selected]

X_train_selected = X_train[colnames_selected]
X_test_selected = X_test[colnames_selected]


In [ ]:
print(colnames_selected)

In [ ]:
print(len(colnames_selected))